In [2]:
import pandas as pd
import numpy as np

In [3]:
categorical_df = pd.read_csv('../dataset/5_categorical_data.csv')
numerical_df = pd.read_csv('../dataset/5_numerical_data.csv')

categorical_df.head()

,formula_pretty,symmetry,structure,decomposes_to,xas,bandstructure,dos,ordering,bulk_modulus
0,O2,"{'crystal_system': 'Monoclinic', 'symbol': 'P2...",c,"[{'material_id': 'mp-611836', 'formula': 'O2',...",NaN,NaN,NaN,NM,NaN
1,W,"{'crystal_system': 'Monoclinic', 'symbol': 'P2...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-91', 'formula': 'W1', 'am...","[{'edge': 'L3', 'absorbing_element': 'W', 'spe...",NaN,NaN,NM,NaN
2,Cu,"{'crystal_system': 'Monoclinic', 'symbol': 'P2...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-30', 'formula': 'Cu1', 'a...","[{'edge': 'L3', 'absorbing_element': 'Cu', 'sp...",NaN,NaN,FM,NaN
3,W,"{'crystal_system': 'Monoclinic', 'symbol': 'P2...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-91', 'formula': 'W1', 'am...","[{'edge': 'L2', 'absorbing_element': 'W', 'spe...",{'setyawan_curtarolo': {'task_id': 'mp-1584057...,"{'total': {'1': {'task_id': 'mp-1709234', 'ban...",NM,NaN
4,Ta,"{'crystal_system': 'Tetragonal', 'symbol': 'P-...","{'@module': 'pymatgen.core.structure', '@class...","[{'material_id': 'mp-569794', 'formula': 'Ta30...","[{'edge': 'K', 'absorbing_element': 'Ta', 'spe...",{'setyawan_curtarolo': {'task_id': 'mp-1624296...,"{'total': {'1': {'task_id': 'mp-1758913', 'ban...",NM,"{'voigt': 195.607, 'reuss': 195.503, 'vrh': 19..."


In [4]:
#begin work on symmetry attribute
import ast

crystal_sytems_df = []

for row in categorical_df.itertuples():
   
    symmetry_dict = ast.literal_eval(row.symmetry) 
    crystal_sytems_df.append(symmetry_dict['crystal_system'])

crystal_sytems_df = pd.get_dummies(crystal_sytems_df, columns=['crystal_system'], 
                                    drop_first=True, prefix='crystal_sys')

crystal_sytems_df = crystal_sytems_df.astype(int)
categorical_df = categorical_df.drop('symmetry', axis=1)

numerical_df = pd.concat([numerical_df, crystal_sytems_df], axis=1)
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49772 entries, 0 to 49771
Data columns (total 43 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   nsites                                        49772 non-null  int64  
 1   nelements                                     49772 non-null  int64  
 2   volume                                        49772 non-null  float64
 3   density                                       49772 non-null  float64
 4   density_atomic                                49772 non-null  float64
 5   energy_per_atom                               48674 non-null  float64
 6   formation_energy_per_atom                     48674 non-null  float64
 7   energy_above_hull                             48674 non-null  float64
 8   is_stable                                     49772 non-null  int64  
 9   equilibrium_reaction_energy_per_atom          23056 non-null 

In [5]:
import ast
import pandas as pd

lattice_matrix_df = []

for row in categorical_df.itertuples(index=False):
    try:
        structure_dict = ast.literal_eval(row.structure)
        
        if 'lattice' in structure_dict and 'matrix' in structure_dict['lattice']:
            lattice_matrix = structure_dict['lattice']['matrix']
            lattice_matrix_df.append(lattice_matrix)

    except (ValueError, SyntaxError) as e:
        lattice_matrix_df.append("none")
lattice_matrix_df = pd.DataFrame(lattice_matrix_df)

print(lattice_matrix_df.head())


                                                   0
0                                               none
1  [[0.0, -2.633443, 0.0], [-9.067128, 0.0, 2.475...
2  [[0.0, -2.2382121, 0.0], [-11.3198294, 0.0, 2....
3  [[9.2328463, 0.0, -0.34462009], [0.0, 2.599867...
4  [[5.30817345, 0.0, 0.0], [0.0, 10.22206745, 0....


In [6]:
#Convert matrices to a single value using Frobeius norm:

def frobenius_norm(matrix_str):
    try:
        matrix = np.array(ast.literal_eval(matrix_str))
        return np.linalg.norm(matrix, 'fro')
    except (ValueError, SyntaxError):
        return None
    

norms_df = []


for row in lattice_matrix_df.itertuples(index=False):
    matrix=row[0]
    norm = frobenius_norm(matrix)
    norms_df.append(norm)

norms_df = pd.DataFrame(norms_df)
print(norms_df)

          0
0      None
1      None
2      None
3      None
4      None
...     ...
49767  None
49768  None
49769  None
49770  None
49771  None

[49772 rows x 1 columns]


In [7]:
numerical_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49772 entries, 0 to 49771
Data columns (total 43 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   nsites                                        49772 non-null  int64  
 1   nelements                                     49772 non-null  int64  
 2   volume                                        49772 non-null  float64
 3   density                                       49772 non-null  float64
 4   density_atomic                                49772 non-null  float64
 5   energy_per_atom                               48674 non-null  float64
 6   formation_energy_per_atom                     48674 non-null  float64
 7   energy_above_hull                             48674 non-null  float64
 8   is_stable                                     49772 non-null  int64  
 9   equilibrium_reaction_energy_per_atom          23056 non-null 